### Pruning & Final Model Selection

Goal: Reduce overfitting by pruning and finalize the best model

Approach:
 - Compare fully grown tree vs. depth-limited tree
 - Apply cost-complexity pruning
 - Perform cross-validation & VC-dimension analysis
 - Evaluate final model on an unseen dataset


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from decision_tree import DecisionTree # Custom implemented Decision Tree 
from sklearn.tree import DecisionTreeClassifier # Sklearn Decision Tree for pruning 
from IPython.display import display
from sklearn.model_selection import cross_val_score


#### Load Processed Dataset

In [2]:
# Load the dataset 
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")
# Load the completely unseen final test set
X_final_test = np.load("X_final_test.npy")
y_final_test = np.load("y_final_test.npy")

#### Train Fully Grown Tree vs. Depth-15 Tree (Before Pruning)

Compare an unpruned fully grown tree with a depth-limited tree to analyze overfitting.
- If the generalization gap is large, the fully grown tree overfits because it memorizes training data.
- The depth-15 tree should perform better on the test set if it generalizes well.

In [3]:
print("Training Fully Grown and Depth-15 Trees...")

# Fully Grown Tree (Overfit)
overfit_tree = DecisionTree(max_depth=None, min_impurity_decrease=0.0, criterion="gini")
overfit_tree.fit(X_train, y_train)

# Depth=15 Tree
depth15_tree = DecisionTree(max_depth=15, min_impurity_decrease=0.0, criterion="gini")
depth15_tree.fit(X_train, y_train)

# Compute Accuracy
overfit_train_acc = accuracy_score(y_train, overfit_tree.predict(X_train))
overfit_test_acc = accuracy_score(y_test, overfit_tree.predict(X_test))

depth15_train_acc = accuracy_score(y_train, depth15_tree.predict(X_train))
depth15_test_acc = accuracy_score(y_test, depth15_tree.predict(X_test))

# Store comparison results
comparison_df = pd.DataFrame({
    "Model": ["Fully Grown (Before Pruning)", "Depth=15 (No Pruning)"],
    "Train Accuracy": [overfit_train_acc, depth15_train_acc],
    "Test Accuracy": [overfit_test_acc, depth15_test_acc],
    "Generalization Gap": [overfit_train_acc - overfit_test_acc, depth15_train_acc - depth15_test_acc]
})

print("Overfitting Analysis:")
display(comparison_df)



Training Fully Grown and Depth-15 Trees...
Overfitting Analysis:


,Model,Train Accuracy,Test Accuracy,Generalization Gap
0,Fully Grown (Before Pruning),1.000000,0.996361,0.003639
1,Depth=15 (No Pruning),0.993818,0.990266,0.003553


#### Cost-Complexity Pruning (Avoid Overfitting)
Reduce overfitting by pruning using different alpha (complexity) values.
- A higher alpha value means stronger pruning, leading to a simpler tree.
- The best alpha is where test accuracy remains stable while train accuracy drops slightly (indicating reduced overfitting).

In [4]:
print("\nPerforming Cost-Complexity Pruning...")

ccp_alpha_values = [0.0001, 0.001, 0.005, 0.01]# Regularization strength

pruning_results = []
for ccp_alpha in ccp_alpha_values:
    sklearn_tree = DecisionTreeClassifier(ccp_alpha=ccp_alpha, criterion="gini")
    sklearn_tree.fit(X_train, y_train)

    train_acc = accuracy_score(y_train, sklearn_tree.predict(X_train))
    test_acc = accuracy_score(y_test, sklearn_tree.predict(X_test))

    pruning_results.append({
        "Alpha": ccp_alpha,
        "Training Accuracy": train_acc,
        "Test Accuracy": test_acc
    })

# Convert results to DataFrame
pruning_results_df = pd.DataFrame(pruning_results)
display(pruning_results_df)



Performing Cost-Complexity Pruning...


,Alpha,Training Accuracy,Test Accuracy
0,0.0001,0.995006,0.994087
1,0.0010,0.963033,0.961426
2,0.0050,0.837642,0.829876
3,0.0100,0.611749,0.608806


#### Final Model Selection (Pruned vs. Depth=15)
Compare the best depth-limited tree vs. pruned tree to decide which is better.
- If the pruned tree performs similarly to the depth-15 tree but has lower overfitting, it’s the better choice.
- Pruning should maintain accuracy while reducing model complexity.

In [5]:
print("\nSelecting Final Model Based on Pruning...")

# Train Pruned Tree (α=0.0001)
pruned_tree = DecisionTreeClassifier(ccp_alpha=0.0001, criterion="gini")
pruned_tree.fit(X_train, y_train)

# Compute Accuracy
pruned_train_acc = accuracy_score(y_train, pruned_tree.predict(X_train))
pruned_test_acc = accuracy_score(y_test, pruned_tree.predict(X_test))


# Store results
final_choices = pd.DataFrame({
    "Model": ["Depth=15 (No Pruning)", "Fully Grown + Light Pruning (α=0.0001)"],
    "Train Accuracy": [depth15_train_acc, pruned_train_acc],
    "Test Accuracy": [depth15_test_acc, pruned_test_acc],
    "Generalization Gap": [depth15_train_acc - depth15_test_acc, pruned_train_acc - pruned_test_acc]
})

display(final_choices)



Selecting Final Model Based on Pruning...


,Model,Train Accuracy,Test Accuracy,Generalization Gap
0,Depth=15 (No Pruning),0.993818,0.990266,0.003553
1,Fully Grown + Light Pruning (α=0.0001),0.994965,0.993905,0.001060


#### Cross-Validation & VC-Dimension Analysis
Ensure the selected model generalizes well using Cross-Validation.
- If CV accuracy matches test accuracy, the model generalizes well and isn’t overfitting.


In [6]:
print("\nPerforming Cross-Validation...")

models = {
    "Depth=15 (No Pruning)": DecisionTree(max_depth=15, min_impurity_decrease=0.0, criterion="gini"),
    "Pruned Fully Grown (α=0.0001)": DecisionTree(max_depth=None, min_impurity_decrease=0.0001, criterion="gini")
}

# Perform 5-fold cross-validation
cv_results = {}

for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    cv_results[name] = scores.mean()  # Store mean accuracy across folds

# Store and display results
import pandas as pd
cv_df = pd.DataFrame(list(cv_results.items()), columns=["Model", "Cross-Validated Accuracy"])
display(cv_df)



Performing Cross-Validation...


,Model,Cross-Validated Accuracy
0,Depth=15 (No Pruning),0.986552
1,Pruned Fully Grown (α=0.0001),0.959984


#### VC-Dimension Estimation (PAC Learning)
Estimate VC-Dimension (Model Complexity).
- If VC-Dimension is too high, the model is too complex and prone to overfitting.
- Lower VC-Dimension suggests better generalization.

In [7]:
print("\nEstimating Model Complexity (VC-Dimension)...")

vc_estimate_15 = X_train.shape[1] * 15  # Features * depth (simplified assumption)
vc_estimate_none = X_train.shape[1] * (np.log2(len(X_train)))  # Approximate for fully grown tree

# Display estimated complexity measures
vc_df = pd.DataFrame({
    "Model": ["Depth=15 (No Pruning)", "Pruned Fully Grown (α=0.0001)"],
    "Estimated VC-Dimension": [vc_estimate_15, vc_estimate_none]
})

display(vc_df)



Estimating Model Complexity (VC-Dimension)...


,Model,Estimated VC-Dimension
0,Depth=15 (No Pruning),225.000000
1,Pruned Fully Grown (α=0.0001),233.643279


#### Out-of-Sample Testing on Final Holdout Set
Evaluate the final model on an unseen dataset.
- If both models perform similarly on unseen data, the pruned tree is preferred because it’s simpler and generalizes better.
- If accuracy drops significantly, the model may still be overfitting, and additional regularization may be needed.


In [8]:
print("\nEvaluating Final Model on Unseen Data...")

final_test_acc_15 = accuracy_score(y_final_test, depth15_tree.predict(X_final_test))
final_test_acc_pruned = accuracy_score(y_final_test, pruned_tree.predict(X_final_test))

# Store final test results
final_test_df = pd.DataFrame({
    "Model": ["Depth=15 (No Pruning)", "Pruned Fully Grown (α=0.0001)"],
    "Final Test Accuracy": [final_test_acc_15, final_test_acc_pruned]
})

display(final_test_df)



Evaluating Final Model on Unseen Data...


,Model,Final Test Accuracy
0,Depth=15 (No Pruning),0.989362
1,Pruned Fully Grown (α=0.0001),0.989362
